In [1]:
import numpy
import cv2
import os
from tqdm import tqdm

In [2]:
path = "../input/dynamic-sign-language-gestures-01/all"
image_size = 256
frame_rate = 0.1

def getFrame(sec):
    vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
    hasFrames,image = vidcap.read()
    if hasFrames:
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = cv2.resize(image, (image_size, image_size))
    return [hasFrames, image]

In [3]:
videos_list = sorted(os.listdir(path))[:500]
new_list = []

for xyz in tqdm(range(10)):
    mno = xyz*50
    for index in range(mno, mno+50):
        new_list.append(videos_list[index])

100%|██████████| 10/10 [00:00<00:00, 18133.61it/s]


In [4]:
frames = []
labels = []

for video in tqdm(new_list):
#     vidcap = cv2.VideoCapture(path + '/' + video)
#     sec = 0
#     mini = []
#     success, frame = getFrame(sec)
#     while success:
#         mini.append(frame)
#         sec = sec + frame_rate
#         sec = round(sec, 2)
#         success, frame = getFrame(sec)
        
#     frames.append(mini[-8:])
    labels.extend([int(video[:3]) -1]*8)

100%|██████████| 500/500 [00:00<00:00, 404777.46it/s]


In [5]:
from tensorflow.keras.utils import to_categorical
# frames = numpy.array(frames)
frames = numpy.load('../input/aslframes/last8-10classes.npy')
labels = numpy.array(labels)
cat_labels = to_categorical(labels)

In [7]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(frames, labels, test_size=0.1,
                                                   shuffle=True, random_state=42, stratify=labels)

In [9]:
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1]*x_train.shape[2]*x_train.shape[3])
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1]*x_test.shape[2]*x_test.shape[3])

In [12]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
print(accuracy_score(y_test, y_pred))
confusion_matrix(y_test, y_pred)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.975


array([[39,  0,  0,  0,  1,  0,  0,  0,  0,  0],
       [ 0, 38,  1,  0,  0,  0,  0,  0,  0,  1],
       [ 0,  1, 39,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0, 40,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0, 39,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  0,  0, 40,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  1,  0, 38,  1,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0, 40,  0,  0],
       [ 1,  0,  0,  0,  0,  0,  0,  0, 39,  0],
       [ 0,  0,  0,  0,  1,  0,  0,  0,  1, 38]])